In [1]:
import numpy as np
import pandas as pd
import itertools
from collections import namedtuple

In [2]:
from utils_s2_tree_search import StemBbTree, BoundingBox, bb_conflict

In [3]:
df = pd.read_pickle('data/data_len40_1000_s1_stem_bb.pkl.gz')


In [4]:
# sanity check with combination computed from brute-force
df_debug = pd.read_pickle('data/data_len40_1000_s1_stem_bb_le10_combos.pkl.gz')

In [5]:
example_id = 0   # ID 0 is the same in both dataset, others are not (second dataset only has those with num_bbs <= 15)

In [6]:
# subset to example with bb sensitivity = 100%
df = df[df['bb_identical'] == 1]

row = df.iloc[example_id]
df_stem = pd.DataFrame(row.pred_stem_bb)

# we use df index, make sure it's contiguous
assert df_stem.iloc[-1].name == len(df_stem) - 1

bbs = {}
for idx, r in df_stem.iterrows():
    bbs[idx] = BoundingBox(bb_x=r['bb_x'],
                           bb_y=r['bb_y'],
                           siz_x=r['siz_x'],
                           siz_y=r['siz_y'])

bb_conf_arr = np.zeros((len(bbs), len(bbs)))
for i in bbs.keys():
    bb1 = bbs[i]
    for j in bbs.keys():
        bb2 = bbs[j]
        # TODO only need to compute half
        bb_conf_arr[i, j] = bb_conflict(bb1, bb2)
assert np.all(bb_conf_arr.T == bb_conf_arr)


In [7]:
len(bbs)

9

In [8]:
bb_tree = StemBbTree(bbs, bb_conf_arr)

In [9]:
len(bb_tree.leaves)

52

In [10]:
combo_ids_tree = []
for x in bb_tree.leaves:
    # convert base2 to base10
    bba_b2 = x.bb_assignment
    bba_b10 = int(''.join([str(int(a)) for a in bba_b2]), 2)
    combo_ids_tree.append(bba_b10)

In [11]:
combo_ids_brute_force = []
for bbi in pd.DataFrame(df_debug.iloc[example_id].valid_combos).bb_inc:
    bba_b2 = np.zeros(len(bbs))
    bba_b2[bbi] = 1
    bba_b10 = int(''.join([str(int(a)) for a in bba_b2]), 2)
    combo_ids_brute_force.append(bba_b10)

In [12]:
assert set(combo_ids_tree) == set(combo_ids_brute_force)